In [22]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Memory hierarchy

## How much time did I spent on "memory"

Remember our demo long time ago?

In [23]:
compare([do_render_code("./madd/madd_A.c", lang="c++", show=["//START", "//END"]),do_render_code("./madd/madd_B.c", lang="c++", show=["//START", "//END"])])

In [24]:
render_code("./mv/matvec.c", lang="c++", show=["//START", "//END"])

// ./mv/matvec.c:32-46 (15 lines)
  //START
    for(i = 0; i < ARRAY_SIZE; i++)
    {
      result = 0.0;    
      for(j = 0; j < ARRAY_SIZE; j++)
      {
        result += a[i][j]*b[j];
        if(dump)
            printf("&a[%d][%d], %p\n&b[%d], %p\n", i, j, &a[i][j], j, &b[j]);
      }
      c[i] = result;
      if(dump)
          printf("&c[%d], %p\n", i, &c[i]);
    }
  //END

In [25]:
! cd mv; make clean; make; ./matvec 16 1 

rm -f matvec
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3  matvec.c -o matvec
&a[0][0], 0x55b98fb5e330
&b[0], 0x55b98fb5ec30
&a[0][1], 0x55b98fb5e338
&b[1], 0x55b98fb5ec38
&a[0][2], 0x55b98fb5e340
&b[2], 0x55b98fb5ec40
&a[0][3], 0x55b98fb5e348
&b[3], 0x55b98fb5ec48
&a[0][4], 0x55b98fb5e350
&b[4], 0x55b98fb5ec50
&a[0][5], 0x55b98fb5e358
&b[5], 0x55b98fb5ec58
&a[0][6], 0x55b98fb5e360
&b[6], 0x55b98fb5ec60
&a[0][7], 0x55b98fb5e368
&b[7], 0x55b98fb5ec68
&a[0][8], 0x55b98fb5e370
&b[8], 0x55b98fb5ec70
&a[0][9], 0x55b98fb5e378
&b[9], 0x55b98fb5ec78
&a[0][10], 0x55b98fb5e380
&b[10], 0x55b98fb5ec80
&a[0][11], 0x55b98fb5e388
&b[11], 0x55b98fb5ec88
&a[0][12], 0x55b98fb5e390
&b[12], 0x55b98fb5ec90
&a[0][13], 0x55b98fb5e398
&b[13], 0x55b98fb5ec98
&a[0][14], 0x55b98fb5e3a0
&b[14], 0x55b98fb5eca0
&a[0][15], 0x55b98fb5e3a8
&b[15], 0x55b98fb5eca8
&c[0], 0x55b98fb5ecc0
&a[1][0], 0x55b98fb5e3c0
&b[0], 0x55b98fb5ec30
&a[1][1], 0x55b98fb5e3c8
&b[1], 0x55b98fb5ec38
&a[1][2], 0x55b98fb5e3d0
&b[2], 0x55b98fb5ec40
&a[1

## The memory hierarchy of my computer

In [26]:
! lscpu | grep "Model name"
! getconf -a | grep CACHE

Model name:                      AMD Ryzen 7 5700G with Radeon Graphics
LEVEL1_ICACHE_SIZE                 32768
LEVEL1_ICACHE_ASSOC                8
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 32768
LEVEL1_DCACHE_ASSOC                8
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  524288
LEVEL2_CACHE_ASSOC                 8
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  16777216
LEVEL3_CACHE_ASSOC                 0
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  0
LEVEL4_CACHE_ASSOC                 0
LEVEL4_CACHE_LINESIZE              0


In [27]:
! ssh htseng@azelf "lscpu | grep 'Model name'; getconf -a | grep CACHE"

Model name:                      12th Gen Intel(R) Core(TM) i9-12900KF
LEVEL1_ICACHE_SIZE                 65536
LEVEL1_ICACHE_ASSOC                8
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 32768
LEVEL1_DCACHE_ASSOC                8
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  2097152
LEVEL2_CACHE_ASSOC                 16
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  31457280
LEVEL3_CACHE_ASSOC                 12
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  0
LEVEL4_CACHE_ASSOC                 0
LEVEL4_CACHE_LINESIZE              0


In [28]:
! ssh htseng@nano "lscpu | grep 'Model name'; getconf -a | grep CACHE"

Model name:          Cortex-A57
LEVEL1_ICACHE_SIZE                 0
LEVEL1_ICACHE_ASSOC                0
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 0
LEVEL1_DCACHE_ASSOC                0
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  0
LEVEL2_CACHE_ASSOC                 0
LEVEL2_CACHE_LINESIZE              0
LEVEL3_CACHE_SIZE                  0
LEVEL3_CACHE_ASSOC                 0
LEVEL3_CACHE_LINESIZE              0
LEVEL4_CACHE_SIZE                  0
LEVEL4_CACHE_ASSOC                 0
LEVEL4_CACHE_LINESIZE              0


## How blocks are stored in a cache?

How these address are stored "if" they're in a direct-mapped, 16B-sized blocks, 16-block cache?

In [29]:
! cd mv; make clean; make; ./matvec 16 1 

rm -f matvec
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3  matvec.c -o matvec
&a[0][0], 0x56281bc8b330
&b[0], 0x56281bc8bc30
&a[0][1], 0x56281bc8b338
&b[1], 0x56281bc8bc38
&a[0][2], 0x56281bc8b340
&b[2], 0x56281bc8bc40
&a[0][3], 0x56281bc8b348
&b[3], 0x56281bc8bc48
&a[0][4], 0x56281bc8b350
&b[4], 0x56281bc8bc50
&a[0][5], 0x56281bc8b358
&b[5], 0x56281bc8bc58
&a[0][6], 0x56281bc8b360
&b[6], 0x56281bc8bc60
&a[0][7], 0x56281bc8b368
&b[7], 0x56281bc8bc68
&a[0][8], 0x56281bc8b370
&b[8], 0x56281bc8bc70
&a[0][9], 0x56281bc8b378
&b[9], 0x56281bc8bc78
&a[0][10], 0x56281bc8b380
&b[10], 0x56281bc8bc80
&a[0][11], 0x56281bc8b388
&b[11], 0x56281bc8bc88
&a[0][12], 0x56281bc8b390
&b[12], 0x56281bc8bc90
&a[0][13], 0x56281bc8b398
&b[13], 0x56281bc8bc98
&a[0][14], 0x56281bc8b3a0
&b[14], 0x56281bc8bca0
&a[0][15], 0x56281bc8b3a8
&b[15], 0x56281bc8bca8
&c[0], 0x56281bc8bcc0
&a[1][0], 0x56281bc8b3c0
&b[0], 0x56281bc8bc30
&a[1][1], 0x56281bc8b3c8
&b[1], 0x56281bc8bc38
&a[1][2], 0x56281bc8b3d0
&b[2], 0x56281bc8bc40
&a[1

In [30]:
!echo "element,address"> addresses.csv; ./mv/matvec 16 1 >> addresses.csv
df = pd.read_csv("addresses.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
block_size = 16
offset_bits = int(math.log2(block_size))
number_of_blocks = 16
index_bits = int(math.log2(number_of_blocks))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % number_of_blocks))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

,element,address,tag,index
0,&a[0][0],0x5579f6d28330,0x5579f6d283,0x3
1,&b[0],0x5579f6d28c30,0x5579f6d28c,0x3
2,&a[0][1],0x5579f6d28338,0x5579f6d283,0x3
3,&b[1],0x5579f6d28c38,0x5579f6d28c,0x3
4,&a[0][2],0x5579f6d28340,0x5579f6d283,0x4
5,&b[2],0x5579f6d28c40,0x5579f6d28c,0x4
6,&a[0][3],0x5579f6d28348,0x5579f6d283,0x4
7,&b[3],0x5579f6d28c48,0x5579f6d28c,0x4
8,&a[0][4],0x5579f6d28350,0x5579f6d283,0x5
9,&b[4],0x5579f6d28c50,0x5579f6d28c,0x5


What if we have a 2-way, 16-byte blocked, 16-block cache?

In [31]:
df = pd.read_csv("addresses.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
block_size = 16
offset_bits = int(math.log2(block_size))
number_of_blocks = 16
way_assoc=2
number_of_sets = number_of_blocks/way_assoc
index_bits = int(math.log2(number_of_sets))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % number_of_blocks))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

,element,address,tag,index
0,&a[0][0],0x5579f6d28330,0xaaf3eda506,0x3
1,&b[0],0x5579f6d28c30,0xaaf3eda518,0x3
2,&a[0][1],0x5579f6d28338,0xaaf3eda506,0x3
3,&b[1],0x5579f6d28c38,0xaaf3eda518,0x3
4,&a[0][2],0x5579f6d28340,0xaaf3eda506,0x4
5,&b[2],0x5579f6d28c40,0xaaf3eda518,0x4
6,&a[0][3],0x5579f6d28348,0xaaf3eda506,0x4
7,&b[3],0x5579f6d28c48,0xaaf3eda518,0x4
8,&a[0][4],0x5579f6d28350,0xaaf3eda506,0x5
9,&b[4],0x5579f6d28c50,0xaaf3eda518,0x5


## Cache performance of code on "real machines"

### NVIDIA Jetson Nano -- Tegra X1

In [32]:
render_code("4way_madd/madd.c", show=["//START","//END"])

// 4way_madd/madd.c:51-56 (6 lines)
  //START
    for(i = 0; i < RUN_ARRAY_SIZE; i++)
    {
        e[i] = (a[i] * b[i] + c[i])/d[i];
    }
  //END

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

#### Run without the 4-way matrix add loop code.

In [33]:
# Run it "without" the above code.
! ssh htseng@nano "lscpu; cd courses/CS203/demo/memory/4way_madd/; make clean madd_nano; valgrind --tool=cachegrind ./madd_nano 16384 0 "

Architecture:        aarch64
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  1
Core(s) per socket:  4
Socket(s):           1
Vendor ID:           ARM
Model:               1
Model name:          Cortex-A57
Stepping:            r1p1
CPU max MHz:         1479.0000
CPU min MHz:         102.0000
BogoMIPS:            38.40
L1d cache:           32K
L1i cache:           48K
L2 cache:            2048K
Flags:               fp asimd evtstrm aes pmull sha1 sha2 crc32
rm -f madd_intel madd_nano *_O3 *~ madd_A_fission cachegrind* *.perf madd_dump
cc -O3 -DHAVE_LINUX_PERF_EVENT_H -g -DNANO madd.c -o madd_nano
==6761== Cachegrind, a cache and branch-prediction profiler
==6761== Copyright (C) 2002-2017, and GNU GPL'd, by Nicholas Nethercote et al.
==6761== Using Valgrind-3.13.0 and LibVEX; rerun with -h for copyright info
==6761== Command: ./madd_nano 16384 0
==6761== 
--6761-- Warning: Cannot auto-detect cache config, using defaults.
--6761--    

Too much detail! Let's use grep to narrow down the outputs.

In [34]:
# Run it "without" the above code.
! ssh htseng@nano "cd courses/CS203/demo/memory/4way_madd/; valgrind --tool=cachegrind ./madd_nano 8192 0 >& nano_without_loop.perf; grep 'D   refs\|D1' nano_without_loop.perf"

==6823== D   refs:      1,527,600  (941,528 rd   + 586,072 wr)
==6823== D1  misses:        8,534  (  2,735 rd   +   5,799 wr)
==6823== D1  miss rate:       0.6% (    0.3%     +     1.0%  )


Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [35]:
! ssh htseng@nano "cd courses/CS203/demo/memory/4way_madd/;valgrind --tool=cachegrind ./madd_nano 8192 8192 >& nano_with_loop.perf; grep 'D   refs\|D1' nano_with_loop.perf"

==6890== D   refs:      1,548,068  (957,904 rd   + 590,164 wr)
==6890== D1  misses:       29,027  ( 19,129 rd   +   9,898 wr)
==6890== D1  miss rate:       1.9% (    2.0%     +     1.7%  )


In [36]:
# Let's do some math here
total_number_of_accesses_before_the_loop = 1527591
total_number_of_accesses_after_the_loop = 1548067
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop = 8530
total_number_of_misses_after_the_loop = 29021
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

access in the loop: 20476 misses in the loop 20491 miss_rate 1.000733


In [37]:
! cd 4way_madd; make madd_dump; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 32768
B = 64
A = 4
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits)%S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

make: *** No rule to make target 'madd_dump'.  Stop.


,element,address,tag,index


### Intel Core i5 9600K -- 8-way L1, 64B-blocked, 32KB cache

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

Let's again dump, parse and simulation the address sequence.

In [46]:
! cd 4way_madd; make madd_dump; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 32768
B = 64
A = 8
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

cc -O3 -DHAVE_LINUX_PERF_EVENT_H -g -DDUMP madd.c -o madd_dump
60.000000


,element,address,tag,index
0,a[0],0x7f469e322000,0x7f469e322,0x0
1,b[0],0x7f469e332000,0x7f469e332,0x0
2,c[0],0x7f469e342000,0x7f469e342,0x0
3,d[0],0x7f469e352000,0x7f469e352,0x0
4,e[0],0x7f469e362000,0x7f469e362,0x0
5,a[1],0x7f469e322008,0x7f469e322,0x0
6,b[1],0x7f469e332008,0x7f469e332,0x0
7,c[1],0x7f469e342008,0x7f469e342,0x0
8,d[1],0x7f469e352008,0x7f469e352,0x0
9,e[1],0x7f469e362008,0x7f469e362,0x0


#### Run without the 4-way matrix add loop code.

In [47]:
# Run it "without" the above code.
! ssh htseng@celebi "lscpu; cd courses/CS203/demo/memory/4way_madd/; rm madd_intel; make madd_intel; valgrind --tool=cachegrind ./madd_intel 16384 0 >& intel_without_loop.perf; grep 'D   refs\|D1' intel_without_loop.perf"

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   39 bits physical, 48 bits virtual
CPU(s):                          6
On-line CPU(s) list:             0-5
Thread(s) per core:              1
Core(s) per socket:              6
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           158
Model name:                      Intel(R) Core(TM) i5-9600K CPU @ 3.70GHz
Stepping:                        12
CPU MHz:                         3700.000
CPU max MHz:                     4600.0000
CPU min MHz:                     800.0000
BogoMIPS:                        7399.70
Virtualization:                  VT-x
L1d cache:                       192 KiB
L1i cache:                       192 KiB
L2 cache:                        1.5 MiB
L3 cache:                    

Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [40]:
# Run it "with" the above code.
! ssh htseng@celebi " cd courses/CS203/demo/memory/4way_madd/; valgrind --tool=cachegrind ./madd_intel 16384 16384  >& intel_with_loop.perf; grep 'D   refs\|D1' intel_with_loop.perf"

==1620256== D   refs:      2,324,926  (1,648,850 rd   + 676,076 wr)
==1620256== D1  misses:       23,747  (   10,784 rd   +  12,963 wr)
==1620256== D1  miss rate:       1.0% (      0.7%     +     1.9%  )


In [41]:
# Let's do some math here
total_number_of_accesses_before_the_loop =2283933
total_number_of_accesses_after_the_loop = 2324899
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop = 13489
total_number_of_misses_after_the_loop = 23741
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

access in the loop: 40966 misses in the loop 10252 miss_rate 0.250256


## Prefetch

x86 provide prefetch instructions. As a programmer, you may insert ```_mm_prefetch``` in x86 programs to perform software prefetch for your code. The gcc compiler also has a flag ```-fprefetch-loop-arrays``` to automatically insert software prefetch instructions.

### Using prefetch in matrix transpose code

The following example is a highly optimized matrix transpose code. In the example, we try to prefetch the next row.

In [42]:
render_code("./prefetch/transpose.cpp", lang="c++", show=["//START", "//END"])

// ./prefetch/transpose.cpp:42-63 (22 lines)
    //START
    //  Iterate each row.
    f_vector *y_iter = T;
    do{
        //  Iterate each column.
        f_vector *ptr_x = y_iter + block;
        f_vector *ptr_y = y_iter + row_size;

        do{

#ifdef ENABLE_PREFETCH
            _mm_prefetch((char*)(ptr_y + row_size),_MM_HINT_T0);
#endif
            swap_block(ptr_x,ptr_y,block);

            ptr_x += block;
            ptr_y += row_size;
        }while (ptr_y < stop_T);

        y_iter += iter_size;
    }while (y_iter < end);
    //END

Now, let's take a look of what's happening!

In [61]:
! echo "Without prefetch -- the baseline"; cs203 job memory "lscpu | grep Model; make prefetch/; ./prefetch/transpose"

Without prefetch -- the baseline
srun -N1 -p cs203m lscpu | grep Model
Model:                           158
Model name:                      Intel(R) Core(TM) i5-9600K CPU @ 3.70GHz
srun -N1 -p cs203m  make prefetch/
srun: job 16104 queued and waiting for resources
srun: job 16104 has been allocated resources
make: Nothing to be done for 'prefetch/'.
srun -N1 -p cs203m  ./prefetch/transpose
srun: job 16105 queued and waiting for resources
srun: job 16105 has been allocated resources
bytes = 4294967296
Starting Data Transpose...   Done
Time: 0.723428 seconds


In [62]:
! echo "With prefetch"; cs203 job memory "./prefetch/transpose_prefetch"

With prefetch
srun -N1 -p cs203m ./prefetch/transpose_prefetch
bytes = 4294967296
Starting Data Transpose...   Done
Time: 0.642476 seconds



Let's try a different machine now.


In [63]:
! cs203 job run "make prefetch/; lscpu | grep Model"
! echo "Without prefetch -- the baseline"; cs203 job run  "./prefetch/transpose"
! echo "With prefetch"; cs203 job run  "./prefetch/transpose_prefetch"

srun -N1 -p cs203 make prefetch/
make: Nothing to be done for 'prefetch/'.
srun -N1 -p cs203  lscpu | grep Model
srun: job 16108 queued and waiting for resources
srun: job 16108 has been allocated resources
Model:                           8
Model name:                      AMD Ryzen 7 2700 Eight-Core Processor
Without prefetch -- the baseline
srun -N1 -p cs203 ./prefetch/transpose
bytes = 4294967296
Starting Data Transpose...   Done
Time: 0.821175 seconds
With prefetch
srun -N1 -p cs203 ./prefetch/transpose_prefetch
bytes = 4294967296
Starting Data Transpose...   Done
Time: 0.650823 seconds
